# Ružička: Authorship Verification in Python

In this notebook, we offer a quick tutorial as to how you could use the code in this repository. While the package is very much geared towards our own work in authorship verification, you might some of the more general functions useful. All feedback and comments are welcome. This code assumes Python 2.7+ (Python 3 has not been tested). You do not need to install the library to run the code below, but please note that there are a number of well-known third-party Python libraries, including:
+ numpy
+ scipy
+ scikit-learn
+ matplotlib
+ seaborn
+ numba

and preferably (for GPU acceleration and/or JIT-compilation):
+ theano
+ numbapro

We recommend installing Continuum's excellent [Anaconda Python framework](https://www.continuum.io/downloads), which comes bundled with most of these dependencies.


## Walk through

By default, we assume that your data sets are stored in a directory the format on the PAN 2014 track on authorship attribution: a directory should minimally include one folder per verification problem (an `unknown.txt` and at least one `known01.txt`) and a `truth.txt`. E.g. for the corpus of Dutch essays (`../data/2014/du_essays/train`), `truth.txt` contains has a tab-separated line with the ground truth for each problem:

```
DE001 Y
DE002 Y
DE003 N
DE004 N
DE005 N
DE006 N
DE007 N
DE008 Y
...
```

To inspect the problems:

In [1]:
ls ../data/2014/du_essays/train

DE001/         DE021/         DE041/         DE061/         DE081/
DE002/         DE022/         DE042/         DE062/         DE082/
DE003/         DE023/         DE043/         DE063/         DE083/
DE004/         DE024/         DE044/         DE064/         DE084/
DE005/         DE025/         DE045/         DE065/         DE085/
DE006/         DE026/         DE046/         DE066/         DE086/
DE007/         DE027/         DE047/         DE067/         DE087/
DE008/         DE028/         DE048/         DE068/         DE088/
DE009/         DE029/         DE049/         DE069/         DE089/
DE010/         DE030/         DE050/         DE070/         DE090/
DE011/         DE031/         DE051/         DE071/         DE091/
DE012/         DE032/         DE052/         DE072/         DE092/
DE013/         DE033/         DE053/         DE073/         DE093/
DE014/         DE034/         DE054/         DE074/         DE094/
DE015/         DE035/         DE055/         DE075/         DE

Let us now load the set of development problems for the Dutch essays:

In [2]:
from ruzicka.utilities import *

D = "../data/2014/du_essays/"
dev_train_data, dev_test_data = load_pan_dataset(D + "train")

This functions loads all documents and splits the development data into a development part (the known documents) and a testing part (the unknown documents). We can unpack these as follows:

In [3]:
dev_train_labels, dev_train_documents = zip(*dev_train_data)
dev_test_labels, dev_test_documents = zip(*dev_test_data)

Let us have a look at the actual test texts:

In [4]:
from __future__ import print_function

for doc in dev_test_documents[:10]:
    print("+ ", doc[:70])

+  ﻿Dankzij het internet zijn we een grote bron aan informatie rijker . A
+  ﻿Het is dus begrijpelijk dat de commerciële zenders meer reclame moete
+  ﻿" Hey , vuile nicht ! Hangt er nog stront aan je lul ? " . Dergelijke
+  ﻿Gelijkheid tussen man en vrouw is iets dat ons al eeuwen in de ban ho
+  ﻿Gisteren was er opnieuw een protest tegen homofilie in de grootstad P
+  ﻿Voetbal is vandaag de dag zonder twijfel de populairste sport in Belg
+  ﻿Door de ongekende groei van nieuwsbronnen en de opkomst van het inter
+  ﻿Woordenboekgebruik uit interesse De categorie woordenboekgebruikers d
+  ﻿Ze bouwden een tegencultuur op die alles verwierp waar hun ouders alt
+  ﻿Als we hier in België op straat rondlopen , merken we dat er zeer vee


For each of these documents we need to decide whether or not they were in fact written by the target authors proposed:

In [5]:
for doc in dev_test_labels[:10]:
    print("+ ", doc[:70])

+  DE001
+  DE002
+  DE003
+  DE004
+  DE005
+  DE006
+  DE007
+  DE008
+  DE009
+  DE010


The first and crucial step is to vectorize the documents using a vector space model. Below, we use generic example, using the 10,000 most common word unigrams and a plain *tf* model:

In [6]:
from ruzicka.vectorization import Vectorizer

vectorizer = Vectorizer(mfi=10000, vector_space="tf", ngram_type="word", ngram_size=1)

dev_train_X = vectorizer.fit_transform(dev_train_documents).toarray()
dev_test_X = vectorizer.transform(dev_test_documents).toarray()

Note that we use `sklearn` conventions here: we fit the vectorizer only on the vocabulary of the known documents and apply it it later to the unknown documents (since in real life too, we will not necessarily know the known documents in advance). This gives us two compatible corpus matrices:

In [7]:
print(dev_train_X.shape)
print(dev_test_X.shape)

(172, 9977)
(96, 9977)


We now encode the author labels in the development problem sets as integers, using sklearn's convenient `LabelEncoder`:

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(dev_train_labels + dev_test_labels)
dev_train_y = label_encoder.transform(dev_train_labels)
dev_test_y = label_encoder.transform(dev_test_labels)
print(dev_test_y)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95]


We now construct and fit an 'O2' verifier: this extrinsic verification technique is based on the General Imposters framework. We apply it with the minmax metric and a profile base, meaning that the known documents for each author will be represented as a mean centroid:

In [9]:
from ruzicka.Order2Verifier import Order2Verifier

dev_verifier = Order2Verifier(
    metric="minmax", base="profile", nb_bootstrap_iter=100, rnd_prop=0.5
)
dev_verifier.fit(dev_train_X, dev_train_y)

We can now obtain the probability which this O1 verifier would assign to each combination of an unknown document and the target author suggested in the problem:

In [10]:
dev_test_scores = dev_verifier.predict_proba(
    test_X=dev_test_X, test_y=dev_test_y, nb_imposters=30
)

	 - # test documents processed: 10 out of 96
	 - # test documents processed: 20 out of 96
	 - # test documents processed: 30 out of 96
	 - # test documents processed: 40 out of 96
	 - # test documents processed: 50 out of 96
	 - # test documents processed: 60 out of 96
	 - # test documents processed: 70 out of 96
	 - # test documents processed: 80 out of 96
	 - # test documents processed: 90 out of 96


This gives us as an array of probability scores for each problem, corresponding to the number of iterations in which the target's author's profile was closer to the anonymous document than to one of the imposters:

In [11]:
print(dev_test_scores)

[0.73 0.55 0.02 0.   0.13 0.05 0.01 1.   0.96 0.78 0.5  0.27 0.95 0.89
 0.01 0.42 0.55 0.   0.02 0.38 0.   0.   0.59 0.31 0.02 0.8  0.01 0.54
 0.02 0.01 0.   0.   0.24 0.   0.   0.19 0.   0.05 0.09 0.   0.2  0.68
 0.04 0.81 1.   0.   0.43 0.03 0.   0.23 0.02 0.46 0.07 0.37 0.71 0.
 0.03 0.69 0.01 0.82 0.97 0.56 0.96 0.47 0.65 0.95 0.42 0.17 0.48 0.89
 0.4  0.01 0.02 0.57 0.09 0.14 0.26 0.   0.98 0.   0.89 0.39 0.   0.
 1.   0.   1.   0.71 0.   0.   0.   0.54 0.   0.01 0.51 0.63]


Let us now load the ground truth to check how well we did:

In [12]:
dev_gt_scores = load_ground_truth(
    filepath=os.sep.join((D, "train", "truth.txt")), labels=dev_test_labels
)
print(dev_gt_scores)

[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]


There is one final step needed: the PAN evaluation measures allow systems to leave a number of difficult problems unanswered, by setting the probability exactly at 0.5. To account for this strict threshold, we fit a score shifter, which will attempt to rectify mid-range score to 0.5. We can tune these parameters as follows:

In [13]:
from ruzicka.score_shifting import ScoreShifter

shifter = ScoreShifter()
shifter.fit(predicted_scores=dev_test_scores, ground_truth_scores=dev_gt_scores)
dev_test_scores = shifter.transform(dev_test_scores)

p1 for optimal combo: 0.15000000000000002
p2 for optimal combo: 0.4100000000000001
AUC for optimal combo: 0.9526909722222222
c@1 for optimal combo: 0.9375


As you can see, this shifter optimizes 2 parameters using a grid search: all values in between *p1* and *p2* will be rectified to 0.5:

In [14]:
print(dev_test_scores)

[0.8407000112533569, 0.7345000070333481, 0.002999999932944775, 0.0, 0.019499999284744267, 0.00750000011175871, 0.0014999999664723875, 1.0, 0.9763999873399734, 0.8701999831199645, 0.7050000000000001, 0.5, 0.9704999929666518, 0.9350999915599822, 0.0014999999664723875, 0.6577999922633171, 0.7345000070333481, 0.0, 0.002999999932944775, 0.5, 0.0, 0.0, 0.7580999845266343, 0.5, 0.002999999932944775, 0.882000007033348, 0.0014999999664723875, 0.7286000126600265, 0.002999999932944775, 0.0014999999664723875, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.00750000011175871, 0.013500000536441806, 0.0, 0.5, 0.8112000042200088, 0.00599999986588955, 0.8879000014066696, 1.0, 0.0, 0.6637000042200089, 0.0044999998994171625, 0.0, 0.5, 0.002999999932944775, 0.6814000049233437, 0.010500000044703485, 0.5, 0.8288999873399734, 0.0, 0.0044999998994171625, 0.8170999985933304, 0.0014999999664723875, 0.8937999957799911, 0.9823000168800353, 0.7404000014066696, 0.9763999873399734, 0.6872999992966652, 0.7934999859333038, 0.970

We can later apply this optimized score shifter to the test problems. Now the main question: how well would our O2 verifier perform on the development problems, given the optimal *p1* and *p2* found? We answer this question using the three evaluation measures used in the PAN competition.

In [15]:
from ruzicka.evaluation import pan_metrics

dev_acc_score, dev_auc_score, dev_c_at_1_score = pan_metrics(
    prediction_scores=dev_test_scores, ground_truth_scores=dev_gt_scores
)
print("Accuracy: ", dev_acc_score)
print("AUC: ", dev_auc_score)
print("c@1: ", dev_c_at_1_score)
print("AUC x c@1: ", dev_auc_score * dev_c_at_1_score)

Accuracy:  0.8645833333333334
AUC:  0.9526909722222222
c@1:  0.9375
AUC x c@1:  0.8931477864583334


Our score shifting approach clearly pays off, since we are able to leave difficult problems unswered, yielding to a higher c@1 than pure accuracy. We can now proceed to the test problems. The following code block runs entire parallel to the approach above: only the score shifter isn't retrained again:

In [16]:
train_data, test_data = load_pan_dataset(D + "test")
train_labels, train_documents = zip(*train_data)
test_labels, test_documents = zip(*test_data)

# vectorize:
vectorizer = Vectorizer(mfi=10000, vector_space="tf", ngram_type="word", ngram_size=1)
train_X = vectorizer.fit_transform(train_documents).toarray()
test_X = vectorizer.transform(test_documents).toarray()

# encode author labels:
label_encoder = LabelEncoder()
label_encoder.fit(train_labels + test_labels)
train_y = label_encoder.transform(train_labels)
test_y = label_encoder.transform(test_labels)

# fit and predict a verifier on the test data:
test_verifier = Order2Verifier(
    metric="minmax", base="profile", nb_bootstrap_iter=100, rnd_prop=0.5
)
test_verifier.fit(train_X, train_y)
test_scores = test_verifier.predict_proba(test_X=test_X, test_y=test_y, nb_imposters=30)

# load the ground truth:
test_gt_scores = load_ground_truth(
    filepath=os.sep.join((D, "test", "truth.txt")), labels=test_labels
)

# apply the optimzed score shifter:
test_scores = shifter.transform(test_scores)

test_acc_score, test_auc_score, test_c_at_1_score = pan_metrics(
    prediction_scores=test_scores, ground_truth_scores=test_gt_scores
)

print("Accuracy: ", test_acc_score)
print("AUC: ", test_auc_score)
print("c@1: ", test_c_at_1_score)
print("AUC x c@1: ", test_auc_score * test_c_at_1_score)

	 - # test documents processed: 10 out of 96
	 - # test documents processed: 20 out of 96
	 - # test documents processed: 30 out of 96
	 - # test documents processed: 40 out of 96
	 - # test documents processed: 50 out of 96
	 - # test documents processed: 60 out of 96
	 - # test documents processed: 70 out of 96
	 - # test documents processed: 80 out of 96
	 - # test documents processed: 90 out of 96
Accuracy:  0.8541666666666666
AUC:  0.9605034722222222
c@1:  0.9071180555555555
AUC x c@1:  0.8712900420765817


While our final test results are a bit lower, the verifier seems to scale reasonably well to the unseen verification problems in the test set.

# First Order Verification

It is interesting now to compare the GI approach to a first-order verification system, which often yields very competitive results too. Our implementation closely resembles the system proposed by Potha and Stamatatos in 2014 (A Profile-based Method for Authorship Verification). We import and fit this O1 verifier:

In [17]:
from ruzicka.Order1Verifier import Order1Verifier

dev_verifier = Order1Verifier(metric="minmax", base="profile")
dev_verifier.fit(dev_train_X, dev_train_y)
dev_test_scores = dev_verifier.predict_proba(test_X=dev_test_X, test_y=dev_test_y)
print(dev_test_scores)

[0.6975731  0.7034548  0.40138882 0.32841128 0.4868254  0.43319505
 0.38204736 0.8891614  0.8856941  0.65019494 0.58696884 0.5480333
 0.8099436  0.71765405 0.24394971 0.5510135  0.3095544  0.18303102
 0.41029483 0.5063761  0.39518255 0.20231014 0.56074613 0.43544942
 0.37015074 0.5194289  0.29052383 0.4464391  0.3594094  0.24937207
 0.31979412 0.29739827 0.36385065 0.2593767  0.29248828 0.4547326
 0.20648855 0.40505153 0.20148689 0.31327838 0.4837969  0.64704436
 0.34279436 0.6349254  0.9198616  0.1382832  0.4663418  0.3931684
 0.26801997 0.24117368 0.3338954  0.35352033 0.36349016 0.21679121
 0.62807065 0.18063825 0.44475883 0.70594865 0.39799708 0.6576137
 0.46410638 0.6396907  0.81281894 0.6116846  0.59626037 0.6942665
 0.55648714 0.38163322 0.6003838  0.7621097  0.11407489 0.
 0.0229103  0.47921854 0.21442336 0.06848055 0.32174557 0.288647
 0.8303501  0.3065713  0.45135087 0.4558156  0.29662842 0.26255327
 0.898336   0.26605827 1.         0.6276396  0.25173336 0.13219732
 0.3100655

Note that in this case, the 'probabilities' returned are only distance-based pseudo-probabilities and don't lie in the range of 0-1. Applying the score shifter is therefore quintessential with O1, since it will scale the distances to a more useful range:

In [18]:
shifter = ScoreShifter()
shifter.fit(predicted_scores=dev_test_scores, ground_truth_scores=dev_gt_scores)
dev_test_scores = shifter.transform(dev_test_scores)
print(dev_test_scores)

p1 for optimal combo: 0.42000000000000004
p2 for optimal combo: 0.49000000000000005
AUC for optimal combo: 0.9006076388888888
c@1 for optimal combo: 0.8752170138888888
[0.8457622939348222, 0.8487619441747666, 0.16858330607414246, 0.13793273806571962, 0.5, 0.5, 0.1604598891735077, 0.9434723180532456, 0.9417039841413499, 0.8215994209051133, 0.7893541079759598, 0.7694969815015793, 0.9030712443590165, 0.8560035651922226, 0.10245887875556947, 0.7710168999433518, 0.1300128471851349, 0.07687302947044374, 0.17232382893562317, 0.7482518047094346, 0.16597667098045352, 0.08497026085853578, 0.7759805279970169, 0.5, 0.1554633128643036, 0.7549087435007096, 0.12202000737190248, 0.5, 0.1509519445896149, 0.10473626732826234, 0.13431352972984315, 0.12490727305412294, 0.15281727433204653, 0.10893821597099305, 0.12284507632255555, 0.5, 0.08672519087791444, 0.17012164235115054, 0.08462449193000794, 0.1315769183635712, 0.5, 0.8199926239252091, 0.1439736306667328, 0.813811966776848, 0.9591294234991075, 0.058

And again, we are now ready to test the performance of O1 on the test problems.

In [19]:
train_data, test_data = load_pan_dataset(D + "test")
train_labels, train_documents = zip(*train_data)
test_labels, test_documents = zip(*test_data)

# vectorize:
vectorizer = Vectorizer(mfi=10000, vector_space="tf", ngram_type="word", ngram_size=1)
train_X = vectorizer.fit_transform(train_documents).toarray()
test_X = vectorizer.transform(test_documents).toarray()

# encode author labels:
label_encoder = LabelEncoder()
label_encoder.fit(train_labels + test_labels)
train_y = label_encoder.transform(train_labels)
test_y = label_encoder.transform(test_labels)

# fit and predict a verifier on the test data:
test_verifier = Order1Verifier(metric="minmax", base="profile")
test_verifier.fit(train_X, train_y)
test_scores = test_verifier.predict_proba(test_X=test_X, test_y=test_y)

# load the ground truth:
test_gt_scores = load_ground_truth(
    filepath=os.sep.join((D, "test", "truth.txt")), labels=test_labels
)

# apply the optimzed score shifter:
test_scores = shifter.transform(test_scores)

test_acc_score, test_auc_score, test_c_at_1_score = pan_metrics(
    prediction_scores=test_scores, ground_truth_scores=test_gt_scores
)

print("Accuracy: ", test_acc_score)
print("AUC: ", test_auc_score)
print("c@1: ", test_c_at_1_score)
print("AUC x c@1: ", test_auc_score * test_c_at_1_score)

Accuracy:  0.8333333333333334
AUC:  0.8895399305555555
c@1:  0.8522135416666666
AUC x c@1:  0.7580779746726706
